In [1]:
import json
import sys
import os
import numpy as np
import skimage
from scipy import ndimage
from scipy import special
from scipy import sparse
import anndata
import nibabel as nib
import pandas as pd
import datashader as ds
import time
from sklearn.decomposition import PCA
import collections
import pickle as pkl
import zarr
import h5py
import anndata as ad

import ants
import nibabel as nib

from dredFISH.Analysis.__init__plots import *
sns.set_style('white')

from dredFISH.Analysis import basicu

sys.path.insert(0, '/home/fangming/projects/dredfish/packages/PySpots')
from MERFISH_Objects.FISHData import *

# Load data

In [2]:
prj_dir = '/bigstore/GeneralStorage/fangming/projects/neural_net/'
dat_dir = prj_dir + 'data_dump/'
# res_dir = prj_dir + 'data_dump/'

In [3]:
# allen matrix
f = f'/bigstore/GeneralStorage/fangming/projects/dredfish/data/rna/scrna_ss_ctxhippo_a_exon_count_matrix_v2.h5ad'
scrna = ad.read_h5ad(f)
print(scrna.shape)

# # DPNMF matrix 
# f = f'{dat_dir}dpnmf.csv' 
# pmat = pd.read_csv(f, index_col=0)
# print(pmat.shape)

# genes
f = '/bigstore/binfo/mouse/Brain/DRedFISH/Allen_V3_Reference' + '/10X/genes.npy' 
tenx_genes = np.load(f)
f = '/bigstore/binfo/mouse/Brain/DRedFISH/Allen_V3_Reference' + '/SmartSeq/genes.npy' 
smrt_genes = np.load(f)
np.all(tenx_genes == smrt_genes)
genes = tenx_genes

f = '/bigstore/GeneralStorage/fangming/projects/neural_net/scripts/results/embmat=max-half_nrml-90000.0-1.27E+05-24-0-2.60E-09-0.01-1.0.json'
with open(f, 'r') as fh:
    embmat = np.array(json.load(fh))
    pmat = pd.DataFrame(embmat, index=genes)
    
pmat.shape

(73347, 45768)


(10861, 24)

In [17]:
# analysis metadata
meta_path = '/bigstore/GeneralStorage/fangming/projects/dredfish/data_dump/analysis_meta_Mar31.json'
# analysis
with open(meta_path, 'r') as fh:
    meta = json.load(fh)

In [4]:
# for each allen cluster, get their DPNMF measurements

# remove genes with zero counts
pmat = pmat[pmat.sum(axis=1) > 0]
# 5818 genes, 5576 overlap with Allen SS 
genes = np.intersect1d(pmat.index.values, scrna.var.index.values)
genes_idx = basicu.get_index_from_array(scrna.var.index.values, genes)
print(np.unique(pmat.index.values).shape, np.unique(scrna.var.index.values).shape, genes.shape)

pmat = pmat.loc[genes]

(2852,) (45768,) (2852,)


In [5]:
%%time
Xcell = scrna.X

# correct for library size (each cluster has the same total counts)
Xc_norm = basicu.sparse_libsize_norm(Xcell)

# projection
Xcr_norm = Xc_norm[:,genes_idx].dot(pmat)

# save as an anndata
adata = ad.AnnData(
    X=Xcr_norm,
    obs=scrna.obs,
)
adata

CPU times: user 1min 29s, sys: 2min 12s, total: 3min 42s
Wall time: 3min 43s


In [11]:
output = f'{dat_dir}/smrt_nn_v1_April26.h5ad'
adata.write(output)
!chmod 444 $output

In [12]:
adata

AnnData object with n_obs × n_vars = 73347 × 24
    obs: 'donor_sex_id', 'donor_sex_label', 'donor_sex_color', 'region_id', 'region_label', 'region_color', 'platform_label', 'cluster_order', 'cluster_label', 'cluster_color', 'subclass_order', 'subclass_label', 'subclass_color', 'neighborhood_id', 'neighborhood_label', 'neighborhood_color', 'class_order', 'class_label', 'class_color', 'exp_component_name', 'external_donor_name_label', 'full_genotype_label', 'facs_population_plan_label', 'injection_roi_label', 'injection_materials_label', 'injection_method_label', 'injection_type_label', 'full_genotype_id', 'full_genotype_color', 'external_donor_name_id', 'external_donor_name_color', 'facs_population_plan_id', 'facs_population_plan_color', 'injection_materials_id', 'injection_materials_color', 'injection_method_id', 'injection_method_color', 'injection_roi_id', 'injection_roi_color', 'injection_type_id', 'injection_type_color', 'cell_type_accession_label', 'cell_type_alias_label', 'cell_